# A/B Testing

## AKA Applied Hypothesis Testing!

If you went through all the stats up to this point and thought "oh man when am I ever going to use this stuff" - I get it. But one of the most common ways that Hypothesis Testing techniques are used in the real world is through A/B Testing!

One of the most common places you see A/B Testing out in the world is in marketing - companies will run A/B tests on elements of their website, their emails, their calls to action, etc. While you see A/B testing in other places, Marketing is going to be my example lens for today's session.

### A/B Testing in Marketing

Hubspot is a marketing software company, and I'm going to use some of their resources in the setup to why all this matters. You can access the specific A/B Testing Kit they put out for marketing optimization process at this link: https://drive.google.com/drive/folders/1Wk3J2nA5gguN1Y_41cACxQ9mcJls9TmI

Hubspot's definition of split testing, aka A/B testing:

> Split testing, commonly referred to as A/B testing, is a method of testing through which marketing variables (such as copy, images, layout, etc) are compared to each other to identify the one that brings a better conversion rate. In this context, the element that is being testing is called the “control” and the element that is argued to give a better result is called the “treatment.”

#### Hubspot's 10 Guidelines for Effective A/B Testing: 

1. Only conduct one test (on one asset) at a time
2. Test one variable at a time
3. Test minor changes, too
4. You can A/B test the entire element
5. Measure as far down funnel as possible
6. Set up control & treatment
7. Decide what you want to test
8. Split your sample group randomly 
9. Test at the same time
10. Decide on necessary significance before testing

### What will the data look like?

Data source: https://www.kaggle.com/zhangluyuan/ab-testing

Unfortunately, this data has no real meta-data associated with it, but the author did say the data comes from an e-commerce website. 

Full credit to Robbie Geoghegan, now a Data Scientist at Facebook, for giving me the idea and sharing work they did on this dataset: https://medium.com/@robbiegeoghegan/implementing-a-b-tests-in-python-514e9eb5b3a1 

Another blog I referenced: https://medium.com/@RenatoFillinich/ab-testing-with-python-e5964dd66143

Before we go any further, and typically before we run a test like this, we need to decide our significance level. Otherwise, let's assume that the group who ran this test did it properly (ran tests in parallel, split users randomly, etc)

Significance Level: $\alpha = .05$

In [1]:
# Imports
import pandas as pd
import numpy as np

from scipy import stats

from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize, proportions_ztest

In [2]:
!ls data/

City_of_Seattle_Wage_Data_Mar2021.csv bikeshare_day.csv
Mushrooms_cleaned.csv                 northern_hemisphere.csv
ab_data.csv                           probability_playlists
avocado.csv                           southern_hemisphere.csv


In [3]:
# Grab our data - want the column 'timestamp' to be a datetime object
df = pd.read_csv('data/ab_data.csv')

In [4]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [6]:
# Check our timeframe
df['timestamp'].min()

'2017-01-02 13:42:05.378582'

In [7]:
df['timestamp'].max()

'2017-01-24 13:41:54.460509'

#### There's an issue...

In [8]:
# Let's see...
df.loc[df['group']== 'control']['landing_page'].value_counts()
#1928 people in the control group got assigned the new page
# Something to always think about and check -- 
#that your control group got assigned the wrong variable

old_page    145274
new_page      1928
Name: landing_page, dtype: int64

In [9]:
df

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


In [10]:
df['user_id'].duplicated().sum()

3894

In [11]:
dupes = df.loc[df['user_id'].duplicated(keep=False)==True]

In [12]:
dupes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7788 entries, 22 to 294355
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       7788 non-null   int64 
 1   timestamp     7788 non-null   object
 2   group         7788 non-null   object
 3   landing_page  7788 non-null   object
 4   converted     7788 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 365.1+ KB


In [ ]:
to_drop_1 = df.loc[df['group'] == 'control') & 

#### One more thing to check...

In [ ]:
# If we remove all duplicates 

#### Now, let's explore:

In [13]:
# Split out our two groups
control_group = df.loc[df['group'] == 'control']
treat_group = df.loc[df['group'] == 'treatment']

In [14]:
# Check the number of samples, timeframe and conv % for each group
for sub_df in [control_group, treat_group]:
    name = list(sub_df['group'])[0].title()
    print(f"Number of Samples in our {name} Group: {len(sub_df):,}")
    print(f"Timeframe: {sub_df['timestamp'].min()} - {sub_df['timestamp'].max()}")
    print(f"Number of Conversions in our {name} Group: {sub_df['converted'].sum():,}")
    print(f"Conversion % in our {name} Group: {sub_df['converted'].mean() * 100:.3f}%")
    print("*"*20)

Number of Samples in our Control Group: 147,202
Timeframe: 2017-01-02 13:42:15.234051 - 2017-01-24 13:41:54.460509
Number of Conversions in our Control Group: 17,723
Conversion % in our Control Group: 12.040%
********************
Number of Samples in our Treatment Group: 147,276
Timeframe: 2017-01-02 13:42:05.378582 - 2017-01-24 13:41:44.097174
Number of Conversions in our Treatment Group: 17,514
Conversion % in our Treatment Group: 11.892%
********************


Our friend at Facebook, whose [blog](https://medium.com/@robbiegeoghegan/implementing-a-b-tests-in-python-514e9eb5b3a1) and [code](https://github.com/RobbieGeoghegan/AB_Testing/blob/master/AB_Testing.ipynb) inspired this notebook, uses two things you can determine in advance to calculate effect size:

> Baseline rate — an estimate of the metric being analyzed before making any changes
> Practical significance level — the minimum change to the baseline rate that is useful to the business, for example an increase in the conversion rate of 0.001% may not be worth the effort required to make the change whereas a 2% change will be

In other words, you can determine the minimum amount of change you want to see between your two groups and use that to calculate effect size (different than calculating effect size after the study has been conducted, which isn't ideal).

To do this with statsmodels, since we're doing a test on a proportion, we use: https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportion_effectsize.html

In [ ]:
# let's grab some useful variables, going ahead and doing for both groups


In [ ]:
# baseline is what we expect given what we have
# here, we'll capture that with our percentage of conversions 
baseline_rate = None
practical_significance = 0.01 # user defined - want at last 1% difference here

effect_size = proportion_effectsize(baseline_rate, baseline_rate + practical_significance)

In [ ]:
# determine our minimum sample size per group
confidence_level = 0.05 # user defined - want to be 95% confident
power = 0.8 # user defined (1 - beta)

min_sample_size = NormalIndPower().solve_power(effect_size = effect_size, 
                                               power = power, 
                                               alpha = confidence_level)

print(f"Required minimum sample size: {min_sample_size:,.0f} per group")

In [ ]:
# Now let's test!
# Using a proportion test (not dealing with means but proportions)


So?

- 
